# Reading some data to feed into the model 

In [1]:
import torch
import imp
import ppuu.costs

from ppuu.data.dataloader import DataStore, Dataset, Normalizer
from ppuu import configs

In [2]:
store = DataStore('/home/us441/nvidia-collab/vlad/traffic-data-5-small/state-action-cost/data_i80_v0/')
ds = Dataset(store, 'train', 20, 30, 40, shift=False, random_actions=False, state_diffs=True)
loader = torch.utils.data.DataLoader(ds, batch_size=2)
normalizer = Normalizer(store.stats)

In [3]:
it = iter(loader)
batch = next(it)

# Loading the model

Found some crazy-ass checkpoint with 86% success rate `/home/us441/nvidia-collab/vlad/results/policy/vanilla_w_no_action_no_diff/seed=8888/checkpoints/epoch=99_sample_step=499990.ckpt` 

In [4]:
path = '/home/us441/nvidia-collab/vlad/results/policy/vanilla_w_no_action_no_diff/seed=8888/checkpoints/epoch=99_sample_step=499990.ckpt'

In [33]:
# from eval_policy.py

from ppuu.lightning_modules.policy import get_module  # noqa

checkpoint = torch.load(path)

Module = get_module('vanilla_v3')

mpur_module = Module(checkpoint["hyper_parameters"])
mpur_module.cuda()
mpur_module._setup_mixout()
mpur_module.load_state_dict(checkpoint["state_dict"], strict=False)
mpur_module.policy_model.diffs = checkpoint["hyper_parameters"][
    "training_config"
]["diffs"]
mpur_module._setup_normalizer(store.stats)

In [6]:
mpur_module

Cls(
  (forward_model): FwdCNNKMNoAction_VAE(
    (encoder): Encoder(
      (f_encoder): Sequential(
        (0): Conv2d(60, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): Dropout2d(p=0.0, inplace=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
        (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (4): Dropout2d(p=0.0, inplace=True)
        (5): LeakyReLU(negative_slope=0.2, inplace=True)
        (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      )
      (s_encoder): Sequential(
        (0): Linear(in_features=100, out_features=256, bias=True)
        (1): Dropout(p=0.0, inplace=True)
        (2): LeakyReLU(negative_slope=0.2, inplace=True)
        (3): Linear(in_features=256, out_features=256, bias=True)
        (4): Dropout(p=0.0, inplace=True)
        (5): LeakyReLU(negative_slope=0.2, inplace=True)
        (6): Linear(in_features=256, out_features=10752, bias=True)
      )
    )
    (d

In [30]:
mpur_module.policy_model.normalizer = normalizer
mpur_module.policy_model.stats = store.stats 
mpur_module.policy_model(batch.conditional_state_seq.with_ego().cuda(), normalize_inputs=False, normalize_outputs=True)

tensor([[ 3.8660211563e+00, -3.4994306043e-04],
        [ 1.5201503754e+01, -4.9233704340e-04]], device='cuda:0')

In [31]:
print(mpur_module.config.training.diffs)

False


In [35]:
checkpoint['hyper_parameters']

{'model_config': {'model_type': 'vanilla_v3',
  'checkpoint': None,
  'forward_model_path': '/home/us441/nvidia-collab/vlad/results/fm/km_no_action/fm_km_no_action_64/seed=42/checkpoints/last.ckpt',
  'n_cond': 20,
  'n_pred': 30,
  'n_hidden': 256,
  'n_feature': 256,
  'n_inputs': 4,
  'n_actions': 2,
  'height': 117,
  'width': 24,
  'h_height': 14,
  'h_width': 3,
  'hidden_size': 10752,
  'turn_power': 1},
 'cost_config': {'u_reg': 0.05,
  'lambda_a': 0.0,
  'lambda_j': 0.0,
  'lambda_l': 0.2,
  'lambda_o': 1.0,
  'lambda_p': 1.0,
  'gamma': 0.99,
  'u_hinge': 0.5,
  'dreaming_z_reg': 0.1,
  'uncertainty_n_pred': 30,
  'uncertainty_n_models': 10,
  'uncertainty_n_batches': 100},
 'training_config': {'learning_rate': 0.0001,
  'n_epochs': 100,
  'n_steps': 500000.0,
  'epoch_size': 500,
  'batch_size': 10,
  'validation_size': 25,
  'validation_period': 1,
  'dataset': '/home/us441/nvidia-collab/vlad/traffic-data-5/state-action-cost/data_i80_v0/',
  'data_shift': False,
  'random_a

# Passing values to the model 

In [29]:
pp = '/home/us441/nvidia-collab/vlad/traffic-data-5/state-action-cost/data_i80_v0/car_sizes.pth'
xx = torch.load(pp)